In [1]:
import pickle
import random
from matplotlib import pyplot as plt
from collections import Counter, namedtuple

token_tuple = namedtuple(
        "token",
        ["index", "text", "lemma", "pos", "tag", "dep", "parent", "parent_index"],
    )

In [2]:
path = "../data/passage_tokens.pkl"
with open(path, "rb") as f:
    passage_tokens = pickle.load(f)
len(passage_tokens)

126799

In [3]:
pid_set = set(passage_tokens.keys())

# Answer Generation

In [4]:
def pass_token(t):
    return t[3] == "PUNCT" or t[3] not in ["NOUN", "VERB", "ADJ", "ADV"] or t[5] == "ROOT"

In [5]:
for pid, sent_dict in passage_tokens.items():
    s_t = []
    for sent_id, sent_tuples in sent_dict.items():
        s_t += sent_tuples
        # print(f"sentence {sent_id+1} in passage {pid}:")
    for t_t in s_t:
        if pass_token(t_t):
            continue
        current_token = token_tuple(*t_t)
        parent_token = token_tuple(*s_t[current_token.parent_index])
        print(f"{parent_token.text}({parent_token.pos}) -{current_token.dep}-> {current_token.text}({current_token.pos})")
    break

definitions(NOUN) -compound-> Ribbit(NOUN)
Discuss(VERB) -dobj-> definitions(NOUN)
with(ADP) -pobj-> community(NOUN)
doing(VERB) -dobj-> best(ADJ)
doing(VERB) -advcl-> make(VERB)
make(VERB) -ccomp-> sure(ADJ)
is(AUX) -nsubj-> content(NOUN)
is(AUX) -acomp-> useful(ADJ)
useful(ADJ) -conj-> accurate(ADJ)
accurate(ADJ) -conj-> safe(ADJ)
by(ADP) -pobj-> chance(NOUN)
chance(NOUN) -acl-> spot(VERB)
comment(NOUN) -amod-> inappropriate(ADJ)
spot(VERB) -dobj-> comment(NOUN)
use(VERB) -advcl-> navigating(VERB)
through(ADP) -pobj-> website(NOUN)
use(VERB) -dobj-> form(NOUN)
use(VERB) -xcomp-> let(VERB)
let(VERB) -ccomp-> know(VERB)
use(VERB) -conj-> take(VERB)
take(VERB) -dobj-> care(NOUN)
take(VERB) -advmod-> shortly(ADV)


# DocT5query

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
t5_path = "../model/docT5query/"

In [7]:
tokenizer = T5Tokenizer.from_pretrained(t5_path)
dt5q = T5ForConditionalGeneration.from_pretrained(t5_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from utils import Data
data_processor = Data(root_path="../")
data_processor.read_in_memory()

正在处理文件collection.sampled.tsv 读取文件的格式为('pid', 'passage')
正在处理文件train_sample_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件train_sample_passv2_qrels.tsv 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件val_2021_53_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件val_2021_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件val_2021.qrels.pass.final.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件test_2022_76_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件test_2022_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件test_2022.qrels.pass.withDupes.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')


In [16]:
prefix = "answer2question"
for pid, q_dict in data_processor.dataset['train_sample_passv2_qrels'].items():
    qid = q_dict['qid']
    print(data_processor.dataset['train_sample_queries'][qid]['query'])
    print(data_processor.dataset['collection.sampled'][pid]['passage'])
    text = prefix + ":" + data_processor.dataset['collection.sampled'][pid]['passage']
    input_ids = tokenizer.encode(text, max_length=384, truncation=True, return_tensors='pt')
    outputs = dt5q.generate(
    input_ids=input_ids,
    max_length=64,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=5)
    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        print(f'{i + 1}: {query}')
    break

)what was the immediate impact of the success of the manhattan project?
'Abstract. The pivotal engineering and scientific success of the Twentieth century was the Manhattan\r\n\tProject. The Manhattan Project assimilated concepts and leaders from all scientific fields\r\n\tand engineering disciplines to construct the first two atomic bombs.'
1: What was the pivotal engineering and scientific success of the twenty-first century was the Manhattan Project?
2: why did the nuclear atom bombs go off first?
3: What is the role of engineering in the Twentieth century?
4: Where can i find an article on the Manhattan Project of the late twentieth century..?
5: The key economic and technological success of the twentieth century was the?
